### feature 대상으로 LGBM 학습  
(Developed from dacon_etri_base_mod1.ipynb)

In [ ]:
from datetime import datetime

# 현재 날짜 및 시간 가져오기
now = datetime.now()
timestamp = now.strftime("%m%d_%H%M")  # 예: 0517_1530


submission_folder = '/users/user/Desktop/dacon/submission/'
submission_file = f'submission_final_mod4_4_1_{timestamp}.csv'


In [ ]:

import numpy as np 
import pandas as pd 
import glob 
import random 
import os 
import matplotlib.pyplot as plt 
import seaborn as sns 
import ast 
import warnings
warnings.filterwarnings('ignore') 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import torch
import tensorflow as tf


In [ ]:
# seed 고정 
SD = 42 
random.seed(SD) 
np.random.seed(SD) 
os.environ['PYTHONHASHSEED'] = str(SD)
tf.random.set_seed(SD)  # TensorFlow 시드 설정

# 파일 경로 설정 - VSCode 상대경로로 변경
# 실제 경로에 맞게 수정 필요
base_folder =  '/users/user/Desktop/ETRI_lifelog_dataset'
folder = '/ch2025_data_items'

data_dir = base_folder + folder 


# Parquet 파일 전체 경로 리스트 
parquet_files = glob.glob(os.path.join(data_dir, 'ch2025_*.parquet')) 

In [ ]:
# 파일 이름을 키로, DataFrame을 값으로 저장할 딕셔너리 
lifelog_data = {} 

# 파일별로 읽기 
for file_path in parquet_files: 
    name = os.path.basename(file_path).replace('.parquet', '').replace('ch2025_', '') 
    lifelog_data[name] = pd.read_parquet(file_path) 
    print(f"✅ Loaded: {name}, shape = {lifelog_data[name].shape}") 

# 딕셔너리에 있는 모든 항목을 독립적인 변수로 할당 
for key, df in lifelog_data.items(): 
    globals()[f"{key}_df"] = df 

In [ ]:
# 메트릭스 파일 읽기
metrics_train = pd.read_csv(base_folder + '/ch2025_metrics_train.csv')
sample_submission = pd.read_csv(base_folder+'/ch2025_submission_sample.csv')

# ✅ 기준 쌍 (subject_id, lifelog_date) 
sample_submission['lifelog_date'] = pd.to_datetime(sample_submission['lifelog_date']) 
test_keys = set(zip(sample_submission['subject_id'], sample_submission['lifelog_date'].dt.date)) 

# ✅ DataFrame 별 timestamp 컬럼 수동 지정 
dataframes = { 
    'mACStatus': (mACStatus_df, 'timestamp'), 
    'mActivity': (mActivity_df, 'timestamp'), 
    'mAmbience': (mAmbience_df, 'timestamp'), 
    'mBle': (mBle_df, 'timestamp'), 
    'mGps': (mGps_df, 'timestamp'), 
    'mLight': (mLight_df, 'timestamp'), 
    'mScreenStatus': (mScreenStatus_df, 'timestamp'), 
    'mUsageStats': (mUsageStats_df, 'timestamp'), 
    'mWifi': (mWifi_df, 'timestamp'), 
    'wHr': (wHr_df, 'timestamp'), 
    'wLight': (wLight_df, 'timestamp'), 
    'wPedo': (wPedo_df, 'timestamp'), 
} 

In [ ]:
# ✅ 분리 함수 
def split_test_train(df, subject_col='subject_id', timestamp_col='timestamp'): 
    df[timestamp_col] = pd.to_datetime(df[timestamp_col], errors='coerce') 
    df = df.dropna(subset=[timestamp_col]) 
    df['date_only'] = df[timestamp_col].dt.date 
    df['key'] = list(zip(df[subject_col], df['date_only'])) 
    test_df = df[df['key'].isin(test_keys)].drop(columns=['date_only', 'key']) 
    train_df = df[~df['key'].isin(test_keys)].drop(columns=['date_only', 'key']) 
    return test_df, train_df 


In [ ]:
# ✅ 결과 저장 
for name, (df, ts_col) in dataframes.items(): 
    print(f"⏳ {name} 분리 중...") 
    test_df, train_df = split_test_train(df.copy(), subject_col='subject_id', timestamp_col=ts_col) 
    globals()[f"{name}_test"] = test_df 
    globals()[f"{name}_train"] = train_df 
    print(f"✅ {name}_test → {test_df.shape}, {name}_train → {train_df.shape}") 

In [ ]:
def process_mACStatus(df): 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    df = df.sort_values(['subject_id', 'timestamp']) 
    results = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        status = group['m_charging'].values # 0/1 상태 
        times = group['timestamp'].values # 충전 상태 비율 
        ratio_charging = status.mean() 
        # 상태 전이 횟수 
        transitions = (status[1:] != status[:-1]).sum() 
        # 연속된 1 상태 길이들 
        lengths = [] 
        current_len = 0 
        for val in status: 
            if val == 1: 
                current_len += 1 
            elif current_len > 0: 
                lengths.append(current_len) 
                current_len = 0 
        if current_len > 0: 
            lengths.append(current_len) 
        avg_charging_duration = np.mean(lengths) if lengths else 0 
        max_charging_duration = np.max(lengths) if lengths else 0 
        results.append({ 
            'subject_id': subj, 
            'date': date, 
            'charging_ratio': ratio_charging, 
            'charging_transitions': transitions, 
            'avg_charging_duration': avg_charging_duration, 
            'max_charging_duration': max_charging_duration, 
        }) 
    return pd.DataFrame(results) 

mACStatus_df2 = process_mACStatus(mACStatus_df) 

In [ ]:
def process_mActivity(df): 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    summary = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        counts = group['m_activity'].value_counts(normalize=True) # 비율 
        row = {'subject_id': subj, 'date': date} 
        # 0~8 비율 저장 
        for i in range(9): 
            row[f'activity_{i}_ratio'] = counts.get(i, 0) 
        # 주요 활동 정보 
        row['dominant_activity'] = group['m_activity'].mode()[0] 
        row['num_unique_activities'] = group['m_activity'].nunique() 
        summary.append(row) 
    return pd.DataFrame(summary) 

mActivity_df2 = process_mActivity(mActivity_df) 

In [ ]:
def enhanced_process_mActivity(df):
    """
    모바일 활동 데이터를 일별로 종합 분석하는 함수
    기존 비율 기반 분석 + MET 값 + 시간대별 패턴 + 활동 강도 분석
    """
    import pandas as pd
    import numpy as np
    
    # 기본 전처리
    df = df.copy()
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['date'] = df['timestamp'].dt.date
    df['hour'] = df['timestamp'].dt.hour
    df['minute'] = df['timestamp'].dt.minute
    
    # MET 값 매핑 (에너지 소비량)
    met_mapping = {
        0: 1.3,  # 가벼운 좌식 활동
        1: 8.0,  # 격렬한 활동
        2: 2.0,  # 추정값 (없었던 활동)
        3: 1.2,  # 매우 가벼운 활동
        4: 3.0,  # 중간 강도 활동
        5: 4.0,  # 추정값
        6: 5.0,  # 추정값
        7: 3.5,  # 중간 강도 활동
        8: 10.0  # 매우 격렬한 활동
    }
    df['met_value'] = df['m_activity'].map(met_mapping).fillna(2.0)
    
    # 활동 강도 분류
    def classify_intensity(activity):
        if activity in [0, 3]:
            return 'low'
        elif activity in [2, 4, 7]:
            return 'moderate'
        elif activity in [1, 5, 6, 8]:
            return 'high'
        else:
            return 'unknown'
    
    df['intensity'] = df['m_activity'].apply(classify_intensity)
    
    summary = []
    
    for (subj, date), group in df.groupby(['subject_id', 'date']):
        # 기존 비율 계산
        counts = group['m_activity'].value_counts(normalize=True)
        row = {'subject_id': subj, 'date': date}
        
        # 1. 기본 활동 비율 (0~8)
        for i in range(9):
            row[f'activity_{i}_ratio'] = counts.get(i, 0)
        
        # 2. 기본 통계
        row['dominant_activity'] = group['m_activity'].mode()[0] if not group['m_activity'].mode().empty else 0
        row['num_unique_activities'] = group['m_activity'].nunique()
        
        # 3. MET 기반 에너지 분석
        row['daily_met_total'] = group['met_value'].sum()
        row['daily_met_mean'] = group['met_value'].mean()
        row['daily_met_max'] = group['met_value'].max()
        row['daily_met_std'] = group['met_value'].std()
        
        # 4. 활동 강도 분석
        intensity_counts = group['intensity'].value_counts(normalize=True)
        row['low_intensity_ratio'] = intensity_counts.get('low', 0)
        row['moderate_intensity_ratio'] = intensity_counts.get('moderate', 0)
        row['high_intensity_ratio'] = intensity_counts.get('high', 0)
        
        # 5. 시간대별 패턴 분석
        # 아침(6-12), 오후(12-18), 저녁(18-24), 밤(0-6)
        morning = group[(group['hour'] >= 6) & (group['hour'] < 12)]
        afternoon = group[(group['hour'] >= 12) & (group['hour'] < 18)]
        evening = group[(group['hour'] >= 18) & (group['hour'] < 24)]
        night = group[group['hour'] < 6]
        
        # 시간대별 주요 활동
        row['morning_dominant'] = morning['m_activity'].mode()[0] if len(morning) > 0 and not morning['m_activity'].mode().empty else -1
        row['afternoon_dominant'] = afternoon['m_activity'].mode()[0] if len(afternoon) > 0 and not afternoon['m_activity'].mode().empty else -1
        row['evening_dominant'] = evening['m_activity'].mode()[0] if len(evening) > 0 and not evening['m_activity'].mode().empty else -1
        row['night_dominant'] = night['m_activity'].mode()[0] if len(night) > 0 and not night['m_activity'].mode().empty else -1
        
        # 시간대별 평균 MET
        row['morning_met_avg'] = morning['met_value'].mean() if len(morning) > 0 else 0
        row['afternoon_met_avg'] = afternoon['met_value'].mean() if len(afternoon) > 0 else 0
        row['evening_met_avg'] = evening['met_value'].mean() if len(evening) > 0 else 0
        row['night_met_avg'] = night['met_value'].mean() if len(night) > 0 else 0
        
        # 6. 활동 변화 패턴
        # 연속된 같은 활동의 길이 (세션 길이)
        activity_changes = (group['m_activity'] != group['m_activity'].shift()).sum()
        row['activity_transitions'] = activity_changes
        row['avg_session_length'] = len(group) / max(activity_changes, 1)
        
        # 7. 고강도 활동 패턴
        high_intensity_activities = group[group['intensity'] == 'high']
        row['high_intensity_sessions'] = len(high_intensity_activities)
        row['max_continuous_high_intensity'] = 0
        
        if len(high_intensity_activities) > 0:
            # 연속된 고강도 활동의 최대 길이
            high_intensity_mask = group['intensity'] == 'high'
            continuous_lengths = []
            current_length = 0
            
            for is_high in high_intensity_mask:
                if is_high:
                    current_length += 1
                else:
                    if current_length > 0:
                        continuous_lengths.append(current_length)
                    current_length = 0
            
            if current_length > 0:
                continuous_lengths.append(current_length)
            
            row['max_continuous_high_intensity'] = max(continuous_lengths) if continuous_lengths else 0
        
        # 8. 활동 다양성 지수 (엔트로피)
        if len(counts) > 1:
            entropy = -sum(counts * np.log2(counts))
            row['activity_entropy'] = entropy
        else:
            row['activity_entropy'] = 0
        
        # 9. 시간대별 활동 비율
        total_records = len(group)
        row['morning_activity_ratio'] = len(morning) / total_records if total_records > 0 else 0
        row['afternoon_activity_ratio'] = len(afternoon) / total_records if total_records > 0 else 0
        row['evening_activity_ratio'] = len(evening) / total_records if total_records > 0 else 0
        row['night_activity_ratio'] = len(night) / total_records if total_records > 0 else 0
        
        # 10. 활동 강도 변화
        if len(group) > 1:
            met_diff = group['met_value'].diff().abs().mean()
            row['met_variability'] = met_diff
        else:
            row['met_variability'] = 0
            
        summary.append(row)
    
    return pd.DataFrame(summary)

# 사용 예시
enhanced_mActivity_df2 = enhanced_process_mActivity(mActivity_df)

In [ ]:
# 지정된 10개 라벨 
top_10_labels = [ 
    "Inside, small room", "Speech", "Silence", "Music", "Narration, monologue", 
    "Child speech, kid speaking", "Conversation", "Speech synthesizer", "Shout", "Babbling" 
] 

def process_mAmbience_top10(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    # 초기화 
    for label in top_10_labels + ['others']: 
        df[label] = 0.0 
    for idx, row in df.iterrows(): 
        parsed = ast.literal_eval(row['m_ambience']) if isinstance(row['m_ambience'], str) else row['m_ambience'] 
        others_prob = 0.0 
        for label, prob in parsed: 
            prob = float(prob) 
            if label in top_10_labels: 
                df.at[idx, label] = prob 
            else: 
                others_prob += prob 
        df.at[idx, 'others'] = others_prob 
    return df.drop(columns=['m_ambience']) 

mAmbience_df2= process_mAmbience_top10(mAmbience_df) 

In [ ]:
def summarize_mAmbience_daily(df): 
    prob_cols = [col for col in df.columns if col not in ['subject_id', 'timestamp', 'date']] 
    # 하루 단위로 평균값 요약 
    daily_summary = df.groupby(['subject_id', 'date'])[prob_cols].mean().reset_index() 
    return daily_summary 

mAmbience_df2 = summarize_mAmbience_daily(mAmbience_df2) 


In [ ]:
def process_mBle(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    features = [] 
    for idx, row in df.iterrows(): 
        entry = ast.literal_eval(row['m_ble']) if isinstance(row['m_ble'], str) else row['m_ble'] 
        rssi_list = [] 
        class_0_cnt = 0 
        class_other_cnt = 0 
        for device in entry: 
            try: 
                rssi = int(device['rssi']) 
                rssi_list.append(rssi) 
                if str(device['device_class']) == '0': 
                    class_0_cnt += 1 
                else: 
                    class_other_cnt += 1 
            except: 
                continue # malformed record 
        feature = { 
            'subject_id': row['subject_id'], 
            'date': row['date'], 
            'device_class_0_cnt': class_0_cnt, 
            'device_class_others_cnt': class_other_cnt, 
            'device_count': len(rssi_list), 
            'rssi_mean': np.mean(rssi_list) if rssi_list else np.nan, 
            'rssi_min': np.min(rssi_list) if rssi_list else np.nan, 
            'rssi_max': np.max(rssi_list) if rssi_list else np.nan, 
        } 
        features.append(feature) 
    return pd.DataFrame(features) 

In [ ]:
def summarize_mBle_daily(df): 
    # row 단위 BLE feature 추출 
    df = process_mBle(df) 
    # 하루 단위로 cnt 합치기 
    grouped = df.groupby(['subject_id', 'date']).agg({ 
        'device_class_0_cnt': 'sum', 
        'device_class_others_cnt': 'sum', 
        'rssi_mean': 'mean', 
        'rssi_min': 'min', 
        'rssi_max': 'max', 
    }).reset_index() 
    # 총합 구해서 비율 계산 
    total_cnt = grouped['device_class_0_cnt'] + grouped['device_class_others_cnt'] 
    grouped['device_class_0_ratio'] = grouped['device_class_0_cnt'] / total_cnt.replace(0, np.nan) 
    grouped['device_class_others_ratio'] = grouped['device_class_others_cnt'] / total_cnt.replace(0, np.nan) 
    # 필요 없는 원래 cnt 컬럼 제거 
    grouped.drop(columns=['device_class_0_cnt', 'device_class_others_cnt'], inplace=True) 
    return grouped 

mBle_df2 = summarize_mBle_daily(mBle_df) 

In [ ]:
def process_mGps(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    features = [] 
    for idx, row in df.iterrows(): 
        gps_list = ast.literal_eval(row['m_gps']) if isinstance(row['m_gps'], str) else row['m_gps'] 
        altitudes = [] 
        latitudes = [] 
        longitudes = [] 
        speeds = [] 
        for entry in gps_list: 
            try: 
                altitudes.append(float(entry['altitude'])) 
                latitudes.append(float(entry['latitude'])) 
                longitudes.append(float(entry['longitude'])) 
                speeds.append(float(entry['speed'])) 
            except: 
                continue 
        features.append({ 
            'subject_id': row['subject_id'], 
            'date': row['date'], 
            'altitude_mean': np.mean(altitudes) if altitudes else np.nan, 
            'latitude_std': np.std(latitudes) if latitudes else np.nan, 
            'longitude_std': np.std(longitudes) if longitudes else np.nan, 
            'speed_mean': np.mean(speeds) if speeds else np.nan, 
            'speed_max': np.max(speeds) if speeds else np.nan, 
            'speed_std': np.std(speeds) if speeds else np.nan, 
        }) 
    return pd.DataFrame(features) 

In [ ]:
m_Gps_df2 = process_mGps(mGps_df) 
m_Gps_df2 = m_Gps_df2.groupby(['subject_id', 'date']).agg({ 
    'altitude_mean': 'mean', 
    'latitude_std': 'mean', 
    'longitude_std': 'mean', 
    'speed_mean': 'mean', 
    'speed_max': 'max', 
    'speed_std': 'mean' 
}).reset_index() 

In [ ]:
def process_mLight(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    df['hour'] = df['timestamp'].dt.hour 
    # 밤(22~05시), 낮(06~21시) 구분 
    df['is_night'] = df['hour'].apply(lambda h: h >= 22 or h < 6) 
    # 하루 단위 요약 
    daily = df.groupby(['subject_id', 'date']).agg( 
        light_mean=('m_light', 'mean'), 
        light_std=('m_light', 'std'), 
        light_max=('m_light', 'max'), 
        light_min=('m_light', 'min'), 
        light_night_mean=('m_light', lambda x: x[df.loc[x.index, 'is_night']].mean()), 
        light_day_mean=('m_light', lambda x: x[~df.loc[x.index, 'is_night']].mean()), 
        light_night_ratio=('is_night', 'mean') # 밤 시간 측정 비율 
    ).reset_index() 
    return daily 

mLight_df2 = process_mLight(mLight_df) 

In [ ]:
def process_mScreenStatus(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    features = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        status = group['m_screen_use'].values 
        ratio_on = status.mean() 
        transitions = (status[1:] != status[:-1]).sum() 
        # 연속된 1 상태 길이들 
        durations = [] 
        current = 0 
        for val in status: 
            if val == 1: 
                current += 1 
            elif current > 0: 
                durations.append(current) 
                current = 0 
        if current > 0: 
            durations.append(current) 
        features.append({ 
            'subject_id': subj, 
            'date': date, 
            'screen_on_ratio': ratio_on, 
            'screen_on_transitions': transitions, 
            'screen_on_duration_avg': np.mean(durations) if durations else 0, 
            'screen_on_duration_max': np.max(durations) if durations else 0, 
        }) 
    return pd.DataFrame(features) 

mScreenStatus_df2 = process_mScreenStatus(mScreenStatus_df) 


In [ ]:
top_apps = [ 
    'One UI 홈', '카카오톡', '시스템 UI', 'NAVER', '캐시워크', 
    '성경일독Q', 'YouTube', '통화', '메시지', '타임스프레드', 'Instagram'
] 

def process_mUsageStats(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    features = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        app_time = {app: 0 for app in top_apps} 
        others_time = 0 
        for row in group['m_usage_stats']: 
            parsed = ast.literal_eval(row) if isinstance(row, str) else row 
            for entry in parsed: 
                app = entry.get('app_name') 
                time = entry.get('total_time', 0) 
                if app in top_apps: 
                    app_time[app] += int(time) 
                else: 
                    others_time += int(time) 
        feature = { 
            'subject_id': subj, 
            'date': date, 
            'others_time': others_time 
        } 
        # 각 앱별 컬럼 추가 
        feature.update({f'{app}_time': app_time[app] for app in top_apps}) 
        features.append(feature) 
    return pd.DataFrame(features) 

mUsageStats_df2 = process_mUsageStats(mUsageStats_df) 

In [ ]:
# 앱 사용 시간 (합계)
mUsageStats_df3 = mUsageStats_df.copy() 
# 방법 1: 기존 DataFrame에 새로운 컬럼 추가
def calculate_total_usage_time(usage_stats):
    """m_usage_stats에서 모든 앱의 total_time 합계 계산"""
    total_time = 0
    for app in usage_stats:
        total_time += app['total_time']
    return total_time

# m_usage_time 컬럼 추가
mUsageStats_df3['m_usage_time'] = mUsageStats_df3['m_usage_stats'].apply(calculate_total_usage_time)

# # 결과 확인
# print("새로운 컬럼이 추가된 DataFrame:")
# print(mUsageStats_df3[['subject_id', 'timestamp', 'm_usage_time']].head())


# # 전체 구조 확인
# print(f"\nDataFrame shape: {mUsageStats_df3.shape}")
# print(f"Columns: {mUsageStats_df3.columns.tolist()}")

mUsageStats_df3 = mUsageStats_df3[['subject_id', 'timestamp', 'm_usage_time']]

# 간단한 방법: 날짜별 사용시간 합계
mUsageStats_df3['date'] = pd.to_datetime(mUsageStats_df3['timestamp']).dt.date

# 날짜별 합계 계산
mUsageStats_df3 = mUsageStats_df3.groupby(['subject_id', 'date'])['m_usage_time'].sum().reset_index()

In [ ]:

def process_mWifi(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    results = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        rssi_all = [] 
        for row in group['m_wifi']: 
            parsed = ast.literal_eval(row) if isinstance(row, str) else row 
            for ap in parsed: 
                try: 
                    rssi = int(ap['rssi']) 
                    rssi_all.append(rssi) 
                except: 
                    continue 
        results.append({ 
            'subject_id': subj, 
            'date': date, 
            'wifi_rssi_mean': np.mean(rssi_all) if rssi_all else np.nan, 
            'wifi_rssi_min': np.min(rssi_all) if rssi_all else np.nan, 
            'wifi_rssi_max': np.max(rssi_all) if rssi_all else np.nan, 
            'wifi_detected_cnt': len(rssi_all) 
        }) 
    return pd.DataFrame(results) 

mWifi_df2 = process_mWifi(mWifi_df) 

In [ ]:

def get_time_block(hour): 
    if 0 <= hour < 6: 
        return 'early_morning' 
    elif 6 <= hour < 12: 
        return 'morning' 
    elif 12 <= hour < 18: 
        return 'afternoon' 
    else: 
        return 'evening' 

def process_wHr_by_timeblock(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    df['block'] = df['timestamp'].dt.hour.map(get_time_block) 
    results = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        block_stats = {'subject_id': subj, 'date': date} 
        for block, block_group in group.groupby('block'): 
            hr_all = [] 
            for row in block_group['heart_rate']: 
                parsed = ast.literal_eval(row) if isinstance(row, str) else row 
                hr_all.extend([int(h) for h in parsed if h is not None]) 
            if not hr_all: 
                continue 
            above_100 = [hr for hr in hr_all if hr > 100] 
            block_stats[f'hr_{block}_mean'] = np.mean(hr_all) 
            block_stats[f'hr_{block}_std'] = np.std(hr_all) 
            block_stats[f'hr_{block}_max'] = np.max(hr_all) 
            block_stats[f'hr_{block}_min'] = np.min(hr_all) 
            block_stats[f'hr_{block}_above_100_ratio'] = len(above_100) / len(hr_all) 
        results.append(block_stats) 
    return pd.DataFrame(results) 

wHr_df2 = process_wHr_by_timeblock(wHr_df) 

In [ ]:

def process_wLight_by_timeblock(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    df['block'] = df['timestamp'].dt.hour.map(get_time_block) 
    results = [] 
    for (subj, date), group in df.groupby(['subject_id', 'date']): 
        block_stats = {'subject_id': subj, 'date': date} 
        for block, block_group in group.groupby('block'): 
            lux = block_group['w_light'].dropna().values 
            if len(lux) == 0: 
                continue 
            block_stats[f'wlight_{block}_mean'] = np.mean(lux) 
            block_stats[f'wlight_{block}_std'] = np.std(lux) 
            block_stats[f'wlight_{block}_max'] = np.max(lux) 
            block_stats[f'wlight_{block}_min'] = np.min(lux) 
        results.append(block_stats) 
    return pd.DataFrame(results) 

wLight_df2 = process_wLight_by_timeblock(wLight_df)

In [ ]:
def process_wPedo(df): 
    df = df.copy() 
    df['timestamp'] = pd.to_datetime(df['timestamp']) 
    df['date'] = df['timestamp'].dt.date 
    summary = df.groupby(['subject_id', 'date']).agg({ 
        'step': 'sum', 
        'step_frequency': 'mean', 
        'distance': 'sum', 
        'speed': ['mean', 'max'], 
        'burned_calories': 'sum' 
    }).reset_index() 
    # 컬럼 이름 정리 
    summary.columns = ['subject_id', 'date', 'step_sum', 'step_frequency_mean', 'distance_sum', 'speed_mean', 'speed_max', 'burned_calories_sum'] 
    return summary 

wPedo_df2 = process_wPedo(wPedo_df) 

In [ ]:

from functools import reduce 
df_list = [ 
    mACStatus_df2, 
    mActivity_df2,
    enhanced_mActivity_df2,  
    mAmbience_df2, 
    mBle_df2, 
    m_Gps_df2, 
    mLight_df2, 
    mScreenStatus_df2, 
    mUsageStats_df2,
    mUsageStats_df3,
    mWifi_df2, 
    wHr_df2, 
    wLight_df2, 
    wPedo_df2 
] 

merged_df = reduce(lambda left, right: pd.merge(left, right, on=['subject_id', 'date'], how='outer'), df_list) 


In [ ]:

# metrics_train의 lifelog_date → datetime.date 형으로 변환 
metrics_train['lifelog_date'] = pd.to_datetime(metrics_train['lifelog_date']).dt.date 

# merged_df의 date도 변환 
merged_df['date'] = pd.to_datetime(merged_df['date']).dt.date 

# 1. date 기준 정렬을 위해 metrics_train의 lifelog_date -> date로 맞추기 
metrics_train_renamed = metrics_train.rename(columns={'lifelog_date': 'date'}) 

# 2. train_df: metrics_train과 일치하는 (subject_id, date) → 라벨 포함 
train_df = pd.merge(metrics_train_renamed, merged_df, on=['subject_id', 'date'], how='inner') 

# 3. test_df: metrics_train에 없는 (subject_id, date) 
merged_keys = merged_df[['subject_id', 'date']] 
train_keys = metrics_train_renamed[['subject_id', 'date']] 
test_keys = pd.merge(merged_keys, train_keys, on=['subject_id', 'date'], how='left', indicator=True) 
test_keys = test_keys[test_keys['_merge'] == 'left_only'].drop(columns=['_merge']) 
test_df = pd.merge(test_keys, merged_df, on=['subject_id', 'date'], how='left') 


In [ ]:

# ✅ 타겟 리스트 
targets_binary = ['Q1', 'Q2', 'Q3', 'S2', 'S3'] 
target_multiclass = 'S1' 

# ✅ feature 준비 
X = train_df.drop(columns=['subject_id', 'sleep_date', 'date', 'Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']) 
X.fillna(0, inplace=True) # 결측값 처리 

test_X = test_df.drop(columns=['subject_id', 'date']) 
test_X.fillna(0, inplace=True) 


In [ ]:

# 컬럼 이름에서 특수 문자 제거/변환 
def sanitize_column_names(df): 
    df.columns = ( 
        df.columns 
        .str.replace(r"[^\w]", "_", regex=True) # 특수문자 → _ 
        .str.replace(r"__+", "_", regex=True) # 연속된 _ 제거 
        .str.strip("_") # 앞뒤 _ 제거 
    ) 
    return df 

# 모든 입력에 적용 
X = sanitize_column_names(X) 
test_X = sanitize_column_names(test_X) 

In [ ]:
# # 결과 저장 
# # 그룹별 중요 특성 선택
# important_features = [
#     # 스크린 관련 특성
#     'screen_on_ratio', 'screen_on_duration_avg', 'screen_on_duration_max',
    
#     # 와이파이 신호 관련 특성
#     'wifi_rssi_max', 'wifi_rssi_mean', 'wifi_detected_cnt',
    
#     # 활동 관련 특성
#     'activity_3_ratio', 'activity_4_ratio',
    
#     # 충전 관련 특성
#     'charging_ratio', 'max_charging_duration', 'avg_charging_duration',
    
#     # 신호 관련 특성
#     'rssi_mean',
    
#     # 빛 관련 특성
#     'light_night_mean', 'light_max', 'light_std',
    
#     # 위치/움직임 관련 특성
#     'altitude_mean', 'speed_max_x',
    
#     # 앱 사용 관련 특성
#     '메시지_time', 'others_time', 'Narration_monologue'
# ]

# # 선택된 특성만으로 데이터셋 구성
# X_selected = X[important_features]
# test_X_selected = test_X[important_features]



In [ ]:
# LightGBM 라이브러리 임포트
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping 

# GridSearch를 위한 교차 검증 설정
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# LightGBM 기본 파라미터 설정
lgbm_params = {
    'n_estimators': 1000,
    'learning_rate': 0.03,
    'random_state': 42,
    'n_jobs': -1,
    'verbosity': -1
}

# 이진 분류 모델 성능 향상을 위한 그리드 서치 파라미터
binary_param_grid = {
    'learning_rate': [0.01, 0.03],
    'n_estimators': [500, 1000],
    'num_leaves': [50, 100],
    'max_depth': [-1, 5],
    'min_child_samples': [10, 30],
    # 'reg_alpha': [0, 0.01, 0.1],
    # 'reg_lambda': [0, 0.01, 0.1],
}

# 다중 분류 모델 성능 향상을 위한 그리드 서치 파라미터
multi_param_grid = {
    'learning_rate': [0.01, 0.03],
    'n_estimators': [500, 1000],
    'num_leaves': [50, 100],
    'max_depth': [-1, 5],
    'min_child_samples': [10, 30],
    # 'reg_alpha': [0, 0.01, 0.1],
    # 'reg_lambda': [0, 0.01, 0.1],
}

In [ ]:
!pip install xgboost
!pip install catboost

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
import warnings
warnings.filterwarnings('ignore')

# ------------------ 앙상블 모델 설정 ------------------
def create_ensemble_models(lgbm_params, n_classes=None):
    """앙상블을 위한 기본 모델들 생성"""
    
    # LightGBM
    if n_classes is None:  # 이진 분류
        lgbm_model = LGBMClassifier(objective='binary', **lgbm_params)
    else:  # 다중 분류
        lgbm_model = LGBMClassifier(objective='multiclass', num_class=n_classes, **lgbm_params)
    
    # XGBoost
    if n_classes is None:  # 이진 분류
        xgb_model = XGBClassifier(
            objective='binary:logistic',
            eval_metric='logloss',
            random_state=42,
            n_estimators=100,
            max_depth=6,
            learning_rate=0.1,
            verbosity=0
        )
    else:  # 다중 분류
        xgb_model = XGBClassifier(
            objective='multi:softprob',
            eval_metric='mlogloss',
            num_class=n_classes,
            random_state=42,
            n_estimators=100,
            max_depth=6,
            learning_rate=0.1,
            verbosity=0
        )
    
    # CatBoost
    if n_classes is None:  # 이진 분류
        cat_model = CatBoostClassifier(
            objective='Logloss',
            random_state=42,
            iterations=100,
            depth=6,
            learning_rate=0.1,
            verbose=False
        )
    else:  # 다중 분류
        cat_model = CatBoostClassifier(
            objective='MultiClass',
            random_state=42,
            iterations=100,
            depth=6,
            learning_rate=0.1,
            verbose=False
        )
    
    return lgbm_model, xgb_model, cat_model

def create_voting_ensemble(lgbm_model, xgb_model, cat_model):
    """Voting 앙상블 생성"""
    voting_ensemble = VotingClassifier(
        estimators=[
            ('lgbm', lgbm_model),
            ('xgb', xgb_model),
            ('cat', cat_model)
        ],
        voting='soft'  # 확률 기반 투표
    )
    return voting_ensemble

In [ ]:
from lightgbm import LGBMClassifier
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 다중 분류 학습 (S1)
y_multi = train_df['S1']

# S1의 클래스 개수 확인
n_classes = len(y_multi.unique())
print(f"S1 클래스 개수: {n_classes}")


# 앙상블 모델 생성
lgbm_model, xgb_model, cat_model = create_ensemble_models(lgbm_params, n_classes=n_classes)
voting_ensemble = create_voting_ensemble(lgbm_model, xgb_model, cat_model)


# 다중 분류 학습 (S1)
model_s1 = voting_ensemble
model_s1.fit(X, y_multi)
multiclass_pred = model_s1.predict(test_X)

# # 특성 중요도 추출 및 데이터프레임 생성
# feature_importance = pd.DataFrame({
#     'feature': X.columns,  # important_features 대신 X.columns 사용
#     'importance': model_s1.feature_importances_
# }).sort_values('importance', ascending=False)

# # 결과 요약 - 특성 중요도 상위 20개 상세 표시
# print("\n===== 주요 특성 중요도 (상위 20개) =====")
# top_20_features = feature_importance.head(20)

# # 인덱스와 함께 특성명과 중요도 값 출력
# for idx, (_, row) in enumerate(top_20_features.iterrows(), 1):
#     print(f"{idx:2d}. {row['feature']:<30} : {row['importance']:.6f}")

# # 전체 특성 중요도 통계
# print(f"\n===== 특성 중요도 통계 =====")
# print(f"전체 특성 개수: {len(feature_importance)}")
# print(f"최대 중요도: {feature_importance['importance'].max():.6f}")
# print(f"최소 중요도: {feature_importance['importance'].min():.6f}")
# print(f"평균 중요도: {feature_importance['importance'].mean():.6f}")
# print(f"상위 20개 중요도 합계: {top_20_features['importance'].sum():.6f}")
# print(f"전체 중요도 대비 상위 20개 비율: {(top_20_features['importance'].sum() / feature_importance['importance'].sum() * 100):.2f}%")

# # 시각화 - 상위 20개만
# plt.figure(figsize=(12, 10))
# sns.barplot(data=top_20_features, x='importance', y='feature', palette='viridis')
# plt.title('상위 20개 특성 중요도', fontsize=16, fontweight='bold')
# plt.xlabel('중요도', fontsize=12)
# plt.ylabel('특성명', fontsize=12)

# # 각 막대에 중요도 값 표시
# for i, (_, row) in enumerate(top_20_features.iterrows()):
#     plt.text(row['importance'] + 0.001, i, f'{row["importance"]:.4f}', 
#              va='center', fontsize=9)

# plt.tight_layout()
# plt.savefig('feature_importance_top20.png', dpi=300, bbox_inches='tight')
# plt.show()

# # 상위 20개 특성명만 별도로 리스트 생성
# top_20_feature_names = top_20_features['feature'].tolist()
# print(f"\n===== 상위 20개 특성명 리스트 =====")
# print(top_20_feature_names)

In [ ]:
# ===== 개별 모델의 feature importance 추출 =====
# 각 모델의 feature importance 추출
lgbm_importance = model_s1.named_estimators_['lgbm'].feature_importances_
xgb_importance = model_s1.named_estimators_['xgb'].feature_importances_
cat_importance = model_s1.named_estimators_['cat'].feature_importances_

# 평균 feature importance 계산
avg_importance = (lgbm_importance + xgb_importance + cat_importance) / 3

# 특성 중요도 데이터프레임 생성
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': avg_importance,
    'lgbm_importance': lgbm_importance,
    'xgb_importance': xgb_importance,
    'cat_importance': cat_importance
}).sort_values('importance', ascending=False)

# 결과 요약 - 특성 중요도 상위 20개 상세 표시
print("\n===== 주요 특성 중요도 (상위 20개) - 앙상블 평균 =====")
top_20_features = feature_importance.head(20)

for idx, (_, row) in enumerate(top_20_features.iterrows(), 1):
    print(f"{idx:2d}. {row['feature']:<30} : {row['importance']:.6f}")

# 개별 모델별 중요도도 표시
print("\n===== 개별 모델별 상위 10개 특성 중요도 =====")
print("\n--- LightGBM ---")
lgbm_top10 = feature_importance.nlargest(10, 'lgbm_importance')
for idx, (_, row) in enumerate(lgbm_top10.iterrows(), 1):
    print(f"{idx:2d}. {row['feature']:<30} : {row['lgbm_importance']:.6f}")

print("\n--- XGBoost ---")
xgb_top10 = feature_importance.nlargest(10, 'xgb_importance')
for idx, (_, row) in enumerate(xgb_top10.iterrows(), 1):
    print(f"{idx:2d}. {row['feature']:<30} : {row['xgb_importance']:.6f}")

print("\n--- CatBoost ---")
cat_top10 = feature_importance.nlargest(10, 'cat_importance')
for idx, (_, row) in enumerate(cat_top10.iterrows(), 1):
    print(f"{idx:2d}. {row['feature']:<30} : {row['cat_importance']:.6f}")

# 전체 특성 중요도 통계
print(f"\n===== 특성 중요도 통계 (앙상블 평균) =====")
print(f"전체 특성 개수: {len(feature_importance)}")
print(f"최대 중요도: {feature_importance['importance'].max():.6f}")
print(f"최소 중요도: {feature_importance['importance'].min():.6f}")
print(f"평균 중요도: {feature_importance['importance'].mean():.6f}")
print(f"상위 20개 중요도 합계: {top_20_features['importance'].sum():.6f}")
print(f"전체 중요도 대비 상위 20개 비율: {(top_20_features['importance'].sum() / feature_importance['importance'].sum() * 100):.2f}%")

# 시각화 - 앙상블 평균 중요도
plt.figure(figsize=(15, 10))

# 서브플롯 생성
fig, axes = plt.subplots(2, 2, figsize=(20, 15))

# 1. 앙상블 평균 중요도 (상위 20개)
sns.barplot(data=top_20_features, x='importance', y='feature', palette='viridis', ax=axes[0,0])
axes[0,0].set_title('앙상블 평균 특성 중요도 (상위 20개)', fontsize=14, fontweight='bold')
axes[0,0].set_xlabel('중요도', fontsize=12)
axes[0,0].set_ylabel('특성명', fontsize=12)



plt.tight_layout()
plt.savefig('ensemble_feature_importance_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

# 상위 20개 특성명 리스트
top_20_feature_names = top_20_features['feature'].tolist()
print(f"\n===== 상위 20개 특성명 리스트 (앙상블 평균 기준) =====")
print(top_20_feature_names)

# 모델 간 중요도 상관관계 분석
print(f"\n===== 모델 간 특성 중요도 상관관계 =====")
corr_lgbm_xgb = np.corrcoef(lgbm_importance, xgb_importance)[0,1]
corr_lgbm_cat = np.corrcoef(lgbm_importance, cat_importance)[0,1]
corr_xgb_cat = np.corrcoef(xgb_importance, cat_importance)[0,1]

print(f"LightGBM vs XGBoost: {corr_lgbm_xgb:.4f}")
print(f"LightGBM vs CatBoost: {corr_lgbm_cat:.4f}")
print(f"XGBoost vs CatBoost: {corr_xgb_cat:.4f}")

In [ ]:
# 선택된 특성만으로 데이터셋 구성
X_selected = X[top_20_feature_names]
test_X_selected = test_X[top_20_feature_names]

In [ ]:
print("\n===== 오토인코더를 사용한 고급 특성 추출 =====")

# 하이퍼파라미터 설정
encoding_dim = min(64, X_selected.shape[1])  # 인코딩 차원 (원본 특성 수보다 작게 설정)
batch_size = 64
epochs = 100

# 입력 차원
input_dim = X_selected.shape[1]

# 오토인코더 모델 구축
input_layer = Input(shape=(input_dim,))

# 인코더 부분
encoded = Dense(128, activation='relu')(input_layer)
encoded = BatchNormalization()(encoded)
encoded = Dropout(0.3)(encoded)
encoded = Dense(64, activation='relu')(encoded)
encoded = BatchNormalization()(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)

# 디코더 부분
decoded = Dense(64, activation='relu')(encoded)
decoded = BatchNormalization()(decoded)
decoded = Dropout(0.3)(decoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = BatchNormalization()(decoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

# 전체 오토인코더 모델
autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# 인코더 부분만 추출 (특성 추출용)
encoder = Model(input_layer, encoded)


In [ ]:
# 훈련/검증 세트 분리
X_train_auto, X_val_auto = train_test_split(X_selected, test_size=0.2, random_state=42)

# 조기 종료 설정
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# 오토인코더 모델 학습
print("오토인코더 학습 중...")
autoencoder.fit(
    X_train_auto, X_train_auto,
    epochs=epochs,
    batch_size=batch_size,
    shuffle=True,
    validation_data=(X_val_auto, X_val_auto),
    callbacks=[early_stopping],
    verbose=1
)

# 인코딩된 특성 추출
print("인코딩된 특성 추출 중...")
encoded_features_train = encoder.predict(X_selected)
encoded_features_test = encoder.predict(test_X_selected)

# 원본 특성과 인코딩된 특성 결합
X_combined = np.hstack([X_selected, encoded_features_train])
test_X_combined = np.hstack([test_X_selected, encoded_features_test])

print(f"원본 특성 수: {X_selected.shape[1]}")
print(f"인코딩된 특성 수: {encoded_features_train.shape[1]}")
print(f"결합된 특성 수: {X_combined.shape[1]}")


In [ ]:
# ------------------ 결합된 특성으로 이진 분류 모델 학습 (앙상블 적용) ------------------
binary_preds_combined = {}
binary_preds_voting = {}
binary_preds_stacking = {}

for col in targets_binary:
    print(f"\n===== {col} 이진 분류 모델 학습 =====")
    y = train_df[col]
    
    # 개별 모델들 생성
    lgbm_model, xgb_model, cat_model = create_ensemble_models(lgbm_params)
    
    # 1. 기존 LightGBM 최적화
    binary_model = LGBMClassifier(objective='binary', **lgbm_params)
    grid_search = GridSearchCV(
        estimator=binary_model,
        param_grid=binary_param_grid,
        scoring='f1',
        cv=cv,
        verbose=1,
        n_jobs=-1
    )
    
    print(f"{col} LightGBM 최적화 중...")
    grid_search.fit(X_combined, y)
    print(f"LightGBM 최적 파라미터: {grid_search.best_params_}")
    print(f"LightGBM 최고 점수: {grid_search.best_score_:.4f}")
    
    best_lgbm = grid_search.best_estimator_
    binary_preds_combined[col] = best_lgbm.predict(test_X_combined)
    
    # 2. Voting 앙상블
    print(f"{col} Voting 앙상블 학습 중...")
    voting_ensemble = create_voting_ensemble(best_lgbm, xgb_model, cat_model)
    voting_ensemble.fit(X_combined, y)
    binary_preds_voting[col] = voting_ensemble.predict(test_X_combined)
    
    # 검증 성능 비교
    X_train_comb, X_val_comb, y_train_bin, y_val_bin = train_test_split(
        X_combined, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # 각 모델 성능 평가
    models = {
        'LightGBM': best_lgbm,
        'Voting': voting_ensemble,
 
    }
    
    print(f"\n--- {col} 모델 성능 비교 ---")
    for name, model in models.items():
        if name == 'LightGBM':
            model_eval = LGBMClassifier(**grid_search.best_params_)
        else:
            model_eval = model
        
        model_eval.fit(X_train_comb, y_train_bin)
        y_pred = model_eval.predict(X_val_comb)
        f1 = f1_score(y_val_bin, y_pred)
        print(f"{name} F1-Score: {f1:.4f}")

In [ ]:
# ------------------ 결합된 특성으로 다중 분류 모델 학습 (앙상블 적용) ------------------
print("\n===== S1 다중 분류 모델 학습 =====")
y_multi = train_df['S1']
n_classes = len(y_multi.unique())

# 개별 모델들 생성
lgbm_multi, xgb_multi, cat_multi = create_ensemble_models(lgbm_params, n_classes)

# 1. 기존 LightGBM 최적화
multiclass_model = LGBMClassifier(objective='multiclass', num_class=n_classes, **lgbm_params)
grid_search_multi = GridSearchCV(
    estimator=multiclass_model,
    param_grid=multi_param_grid,
    scoring='f1_macro',
    cv=cv,
    verbose=1,
    n_jobs=-1
)

print("LightGBM 다중 분류 최적화 중...")
grid_search_multi.fit(X_combined, y_multi)
print(f"LightGBM 최적 파라미터: {grid_search_multi.best_params_}")
print(f"LightGBM 최고 점수: {grid_search_multi.best_score_:.4f}")

best_lgbm_multi = grid_search_multi.best_estimator_
multiclass_pred_combined = best_lgbm_multi.predict(test_X_combined)

# 2. Voting 앙상블 (다중 분류)
print("Voting 앙상블 다중 분류 학습 중...")
voting_multi = create_voting_ensemble(best_lgbm_multi, xgb_multi, cat_multi)
voting_multi.fit(X_combined, y_multi)
multiclass_pred_voting = voting_multi.predict(test_X_combined)


In [ ]:
# ------------------ 성능 평가 및 비교 ------------------
X_train_comb, X_val_comb, y_train_multi, y_val_multi = train_test_split(
    X_combined, y_multi, test_size=0.2, random_state=42, stratify=y_multi
)

# 다중 분류 모델들 성능 비교
multi_models = {
    'LightGBM': best_lgbm_multi,
    'Voting': voting_multi,

}

print("\n===== 다중 분류 모델 성능 비교 =====")
for name, model in multi_models.items():
    if name == 'LightGBM':
        model_eval = LGBMClassifier(**grid_search_multi.best_params_)
    else:
        model_eval = model
    
    model_eval.fit(X_train_comb, y_train_multi)
    y_pred_multi = model_eval.predict(X_val_comb)
    
    print(f"\n--- {name} 성능 ---")
    print(classification_report(y_val_multi, y_pred_multi))
    print("혼동 행렬:")
    print(confusion_matrix(y_val_multi, y_pred_multi))

In [ ]:
# ------------------ 성능 평가 및 비교 ------------------
X_train_comb, X_val_comb, y_train_multi, y_val_multi = train_test_split(
    X_combined, y_multi, test_size=0.2, random_state=42, stratify=y_multi
)

# 다중 분류 모델들 성능 비교
multi_models = {
    'LightGBM': best_lgbm_multi,
    'Voting': voting_multi,

}

print("\n===== 다중 분류 모델 성능 비교 =====")
for name, model in multi_models.items():
    if name == 'LightGBM':
        model_eval = LGBMClassifier(**grid_search_multi.best_params_)
    else:
        model_eval = model
    
    model_eval.fit(X_train_comb, y_train_multi)
    y_pred_multi = model_eval.predict(X_val_comb)
    
    print(f"\n--- {name} 성능 ---")
    print(classification_report(y_val_multi, y_pred_multi))
    print("혼동 행렬:")
    print(confusion_matrix(y_val_multi, y_pred_multi))

In [ ]:
# # ------------------ 성능 비교 분석 ------------------

# # 원본 특성만 사용한 모델과 결합 특성 모델 비교
# print("\n===== 원본 특성 vs 결합 특성 모델 성능 비교 =====")

# # 원본 특성 모델 재학습 (비교용)
# X_train_orig, X_val_orig, y_train, y_val = train_test_split(
#     X_selected, y_multi, test_size=0.2, random_state=42, stratify=y_multi
# )

# eval_model_orig = LGBMClassifier(**grid_search_multi.best_params_)
# eval_model_orig.fit(X_train_orig, y_train)
# y_pred_orig = eval_model_orig.predict(X_val_orig)

# from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# # 정확도 비교
# acc_orig = accuracy_score(y_val, y_pred_orig)
# acc_comb = accuracy_score(y_val, y_pred_comb)
# print(f"정확도 - 원본 특성: {acc_orig:.4f}, 결합 특성: {acc_comb:.4f}, 향상: {(acc_comb-acc_orig)*100:.2f}%")

# # F1 점수 비교 (다중 분류는 macro 평균)
# f1_orig = f1_score(y_val, y_pred_orig, average='macro')
# f1_comb = f1_score(y_val, y_pred_comb, average='macro')
# print(f"F1 점수 - 원본 특성: {f1_orig:.4f}, 결합 특성: {f1_comb:.4f}, 향상: {(f1_comb-f1_orig)*100:.2f}%")


In [ ]:
# # 검증 데이터에 대한 성능 평가
# X_train_sel, X_val_sel, y_train, y_val = train_test_split(
#     X_selected, y_multi, test_size=0.2, random_state=42, stratify=y_multi
# )

# # 최적 파라미터를 사용하여 검증용 모델 학습
# eval_model_sel = LGBMClassifier(**grid_search_multi.best_params_)
# eval_model_sel.fit(X_train_sel, y_train)

# # 예측 및 평가
# y_pred_sel = eval_model_sel.predict(X_val_sel)

# print("\n===== LightGBM 최적화 모델 평가 (S1) =====")
# print(classification_report(y_val, y_pred_sel))
# print("\n혼동 행렬:")
# print(confusion_matrix(y_val, y_pred_sel))

# # 성능 평가 (원본 모델과 비교)
# # 검증 데이터 분할
# X_train_sel, X_val_sel, y_train, y_val = train_test_split(
#     X_selected, y_multi, test_size=0.2, random_state=42, stratify=y_multi
# )

# # 평가용 모델 학습
# eval_model_sel = LGBMClassifier(**multiclass_params)
# eval_model_sel.fit(X_train_sel, y_train)

# # 예측 및 평가
# y_pred_sel = eval_model_sel.predict(X_val_sel)

# print("\n===== LightGBM 그룹 기반 특성 선택 모델 평가 (S1) =====")
# print(classification_report(y_val, y_pred_sel))

In [ ]:
# sample 기반 제출 포맷 가져오기
submission_final = sample_submission[['subject_id', 'sleep_date', 'lifelog_date']].copy()

# lifelog_date 기준으로 string → date 형식 통일
submission_final['lifelog_date'] = pd.to_datetime(submission_final['lifelog_date']).dt.date

# ID 만들기 (submission에서 예측한 결과와 연결하기 위해)
submission_final['ID'] = submission_final['subject_id'] + '_' + submission_final['lifelog_date'].astype(str)

In [ ]:

# 예측 결과 연결할 수 있도록 동일한 순서로 정렬
# 보통 예측 결과는 test_df 기준이므로 정렬 보장되어야 함
assert len(submission_final) == len(multiclass_pred_voting) # shape 체크

# 다중 분류 예측 붙이기
submission_final['S1'] = multiclass_pred_voting

# 이진 분류 결과 붙이기
for col in ['Q1', 'Q2', 'Q3', 'S2', 'S3']:
    submission_final[col] = binary_preds_voting[col].astype(int) # 확률 아닌 class 예측


In [ ]:

# 최종 제출 형식 정렬
submission_final = submission_final[['subject_id', 'sleep_date', 'lifelog_date', 'Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3']]

# 저장
submission_final.to_csv(submission_folder + submission_file, index=False)

# VSCode에서는 files.download()가 작동하지 않으므로 대체

print(f"✅ 제출 파일이 저장되었습니다: {os.path.abspath(submission_file)}")